# DEEP LEARNING

## Objetivo:

Aplique los temas vistos en los seminarios de Deep Learning en sus proyectos de investigación y familiarícese con librerías como word2vec y keras.

## Trabajo Grupal
### Integrantes
* Carlos Alfredo Silva Villafuerte

* pip install Cython
* pip install word2vec
* pip	install	unidecode	
* pip	install	nltk	
* pip	install	stop_words	
* pip	install	sklearn	
* pip	install	scipy	
* pip	install	matplotlib	


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from numpy import genfromtxt

from unidecode import unidecode
from nltk.tokenize import TweetTokenizer 
from stop_words import get_stop_words
import nltk
from string import punctuation 
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
## 1) Read

In [ ]:
file_datapath = "Publicaciones.csv"
file_datapathTxt = "Publicaciones.txt"
file_datapathTxtWithoutNoise = "Publicaciones_without_Noise.txt"
df = pd.read_csv(file_datapath, sep=',', comment='#' , error_bad_lines=True )


In [ ]:
## Delete noise

In [ ]:
tknzr = TweetTokenizer()
stop_words = [unidecode(stopW) for stopW in stopwords.words('spanish')]

# add aditional stop words
non_words = list(punctuation)
non_words.extend(['¿', '¡', 'q', 'd', 'x', 'xq', '...', '..'])
stop_words = stop_words + non_words

In [ ]:
##Recorer cada publicacion
dirty_sentences = np.array(df['publicacion'])
#dirty_sentences = dirty_sentences[np.logical_not(np.isnan(dirty_sentences))]

#Clean yor sentences and save them in another list
clean_sentences=[] #list to put previous sentences once they are preprocessed
for dirty_sent in dirty_sentences:
    #dirty_sent = str(dirty_sent).encode('utf-8')
    #print(dirty_sent)
    tokens=[token.lower() for token in tknzr.tokenize(unidecode(dirty_sent)) 
                          if token.lower() not in stop_words]
    processed_sentence = ' '.join(tokens) 
    clean_sentences.append(processed_sentence)
np.savetxt(file_datapathTxtWithoutNoise, clean_sentences, delimiter=",")

In [ ]:
## 1) Create a file only with the publications titles

In [ ]:
df['publicacion'].to_csv(file_datapathTxt, header=True, index=False, sep='\t', mode='a')

In [ ]:
## 2) Create a model

In [ ]:
import word2vec 
word2vec.word2vec(file_datapathTxt, 'Publicaciones_model.bin', size=300, window= 5, verbose=True)

In [ ]:
model = word2vec.load('Publicaciones_model.bin') #load the trained model
print(model.vectors.shape)

In [ ]:
indexes, metrics = model.similar('estudio',10)  
model.generate_response(indexes, metrics).tolist()

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
model.vocab[10:20]